In [13]:
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.worksheet.table import Table
from openpyxl.utils import range_boundaries, get_column_letter

workbook = Path("test_workbook.xlsx")
assert workbook.is_file()

In [17]:
def make_ref(min_col, min_row, max_col, max_row):
    return f"{get_column_letter(min_col)}{min_row}:{get_column_letter(max_col)}{max_row}"    

def append_table_values(ws, table, row_values: dict):
    # Maybe later expand funtion to insert values above total rows if they exist!
    if table.totalsRowShown:
        raise RuntimeError(
            "append_table_values() cannot be used on tables with a totals row; "
            "this requires row insertion logic."
        )
    
    columns = { col: i for i, col in enumerate(table.column_names) }
    min_col, min_row, max_col, max_row = range_boundaries(table.ref)
    row_i = max_row + 1
    
    if any(ws.cell(row=row_i, column=c).value is not None for c in range(min_col, max_col+1)):
        raise ValueError(f"Row {row_i} is not empty in the table area; would overwrite data!")
    
    for col, val in row_values.items():
        if col not in columns:
            raise KeyError(f"{col} is not an existing column name!")
        col_i = min_col + columns[col]
        ws.cell(row=row_i, column=col_i, value=val)

    new_ref = make_ref(min_col, min_row, max_col, row_i)
    table.ref = new_ref

A2:G22
(1, 2, 7, 22)


'A2:G22'

In [18]:
wb = load_workbook(workbook)
ws = wb["Running sessions"]
tbl: Table = ws.tables["tblRun"]

ref = tbl.ref
print(ref)
boundaries = range_boundaries(ref)
print(boundaries)
make_ref(*boundaries)
# tbl.column_names

A2:G22
(1, 2, 7, 22)


'A2:G22'